In [4]:
from collections import Counter
import os
import pickle
import glob

import matplotlib.ticker as tkr
from IPython.core.display import HTML
import pandas as pd
import scipy.spatial
import scipy.cluster
import seaborn as sns

from gscripts.general import parsers
from gscripts.general.dataviz import Figure
from gscripts.GO import GO
import gffutils
from scipy.stats import hypergeom

img_path = "/nas3/gpratt/Dropbox/Presentations/notebooks/public_clip"

In [18]:
!mkdir -p /nas3/gpratt/projects/public_clip/analysis/v1_small/

In [85]:
!scp tscc-login.sdsc.edu:/home/gpratt/projects/public_clip/analysis/v1_small/*.{metrics,*.final.out,peaks.bed} /nas3/gpratt/projects/public_clip/analysis/v1_small/

rep1.merged.peaks.metrics                     100%   20     0.0KB/s   00:00    
SRR644273.polyATrim.adapterTrim.metrics       100%   11KB  10.6KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rep.sorted.rm 100%   57     0.1KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rmRep.metrics 100%   39KB  39.4KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rmRep.NRF.met 100%   39     0.0KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rmRep.rmDup.m 100%   57     0.1KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rmRep.rmDup.s 100%   21     0.0KB/s   00:00    
SRR644273.polyATrim.adapterTrim.rmRep.rmDup.s 100%   21     0.0KB/s   00:00    
SRR644274.polyATrim.adapterTrim.metrics       100%   12KB  11.5KB/s   00:00    
SRR644274.polyATrim.adapterTrim.rep.sorted.rm 100%   57     0.1KB/s   00:00    
SRR644274.polyATrim.adapterTrim.rmRep.metrics 100%   41KB  41.1KB/s   00:00    
SRR644274.polyATrim.adapterTrim.rmRep.NRF.met 100%    0     0.0KB/s   00:00    
SRR644274.polyATrim.adapterTrim.rmRep.rm

In [5]:
clip_rnaseq_df = parsers.clipseq_metrics("/nas3/gpratt/projects/public_clip/analysis/v1_small/", iclip=False)
public_clip_list = pd.read_csv("/nas3/gpratt/projects/public_clip/v1/public_clip_list.csv", index_col=0)
clip_rnaseq_df = pd.merge(clip_rnaseq_df, public_clip_list, left_index=True, right_index=True, how="outer")
clip_df = clip_rnaseq_df[clip_rnaseq_df['CLIP-seq?'] == "Yes"]

In [6]:
clip_rnaseq_df["Reads After Triming"] = clip_rnaseq_df['Input Reads'] - clip_rnaseq_df['Too short reads']
clip_rnaseq_df["Fraction Collapsed"] = clip_rnaseq_df['Usable Reads'] / clip_rnaseq_df['Uniquely Mapped Reads'].astype(float)
clip_rnaseq_df["Fraction Usable"] = clip_rnaseq_df['Usable Reads'] / clip_rnaseq_df['Input Reads'].astype(float)
clip_df = clip_rnaseq_df[clip_rnaseq_df['CLIP-seq?'] == "Yes"]

In [7]:
taf15_clips = """SRR644277
SRR644276
SRR644275
SRR644274
SRR644273""".split()

#SRR070463
#SRR070462

In [8]:
taf15_clip_df = clip_df.ix[taf15_clips]

In [9]:
filtered_df = taf15_clip_df[[ "Input Reads",
                             "Too short reads",
                             "Reads that were too short percent",
                             'repetitive_count',
                        "Reads Passing Quality Filter", 
                        "Uniquely Mapped Reads",
                        "Uniquely mapped reads %",
                        "Usable Reads",
                            
                             
                            #"spot",
                            "Num Peaks",
                            "Fraction Collapsed",
                             "Fraction Usable"                             
                            ]]


HTML(filtered_df.to_html(formatters={"Input Reads" : parsers.commas,
                                     'repetitive_count': parsers.commas,
                                     "Too short reads": parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

,Input Reads,Too short reads,Reads that were too short percent,repetitive_count,Reads Passing Quality Filter,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Num Peaks,Fraction Collapsed,Fraction Usable
SRR644277,"1,435,247","948,376",66.1,"258,852","398,658","269,806",67.68%,"38,190",76,0.141546,0.026609
SRR644276,"6,775,718","1,306,069",19.3,"2,675,443","4,491,207","2,447,184",54.49%,"94,666","1,814",0.038684,0.013971
SRR644275,"20,006,057","9,554,752",47.8,"5,490,808","8,369,797","4,816,707",57.55%,"224,393","7,743",0.046586,0.011216
SRR644274,"16,842,608","4,173,889",24.8,"6,822,491","9,507,813","5,186,869",54.55%,"147,253",0,0.028390,0.008743
SRR644273,"3,017,908","978,299",32.4,"1,377,550","1,566,891","1,060,850",67.70%,"43,344",565,0.040858,0.014362


In [10]:
filtered_df["Input Reads"].sum()

48077538.0

In [11]:
filtered_df["Uniquely Mapped Reads"].sum()

13781416.0

In [12]:
#My numbers
23634365  / filtered_df["Usable Reads"].sum()

43.140526717362178

In [14]:
#Kasey's numbers
5975094 / filtered_df["Usable Reads"].sum()

10.906521175658852

In [ ]:
2,099,088